In [1]:
import pandas as pd
import random
import csv
import datetime
from pandas_datareader import data

In [2]:
dataset = pd.read_csv('companylist.csv')

In [3]:
dataset = dataset.loc[:, ~dataset.columns.str.contains('^Unnamed')]

In [4]:
dataset.drop(['Summary Quote','LastSale','ADR TSO','IPOyear'],axis=1, inplace=True)

In [5]:
dataset = dataset[(dataset['MarketCap'] > 0)]

In [6]:
def classifyL2(row):
    if (row['MarketCap'] >= 10000000000):
        val = "Large Cap"
    elif (row['MarketCap'] < 10000000000) & (row['MarketCap'] >= 2000000000):
        val = "Mid Cap"
    elif (row['MarketCap'] < 2000000000) & (row['MarketCap'] >= 300000000):
        val = "Small Cap"
    else:
        val = "Micro Cap"
    return val

In [7]:
dataset['L2'] = dataset.apply(classifyL2, axis=1)

In [8]:
dataset['L2'].value_counts()

Micro Cap    1628
Small Cap     954
Mid Cap       476
Large Cap     215
Name: L2, dtype: int64

In [9]:
dataset.loc[dataset['Symbol'].isin(['AAPL'])]

,Symbol,Name,MarketCap,Sector,Industry,L2
247,AAPL,Apple Inc.,1.397321e+12,Technology,Computer Manufacturing,Large Cap


In [10]:
dataset['risk_score'] = dataset.apply(lambda row: round(random.uniform(1, 18),2), axis=1)

In [11]:
dataset.head()

,Symbol,Name,MarketCap,Sector,Industry,L2,risk_score
0,TXG,"10x Genomics, Inc.",8.042693e+09,Capital Goods,Biotechnology: Laboratory Analytical Instruments,Mid Cap,11.27
1,YI,"111, Inc.",1.342119e+08,Health Care,Medical/Nursing Services,Micro Cap,4.55
2,PIH,"1347 Property Insurance Holdings, Inc.",3.428200e+07,Finance,Property-Casualty Insurers,Micro Cap,16.91
3,PIHPP,"1347 Property Insurance Holdings, Inc.",1.869700e+07,Finance,Property-Casualty Insurers,Micro Cap,16.82
4,TURN,180 Degree Capital Corp.,6.722257e+07,Finance,Finance/Investors Services,Micro Cap,3.21


In [12]:
from yahoo_fin.stock_info import *
import numpy as np

In [13]:
def getClosePrice(row):
    # We would like all available data from 01/01/2000 until 12/31/2016.
    #start_date = datetime.datetime.today()
    #end_date = datetime.datetime.today()
    #print(row['Symbol'])
    # User pandas_reader.data.DataReader to load the desired data. As simple as that.
    #panel_data = data.DataReader(row['Symbol'], 'yahoo', '2020-01-24') #end_date.strftime('%Y-%m-%d')
    info = 0
    try:
        info = get_live_price(row['Symbol'])
    except Exception:
        pass
    return round(info,4)

In [25]:
def computeReturns(row):
    return_series = pd.Series([])
    try:
        quote = data.get_data_yahoo(row['Symbol'],'01/01/2009',interval='d')
        #daily_returns = quote['Adj Close'].pct_change()
        #monthly_returns = quote['Adj Close'].resample('M').ffill().pct_change()
        yearly_returns = quote['Adj Close'].resample('Y').ffill().pct_change()
        
        return_series = pd.Series([yearly_returns['2019-12-31'],yearly_returns['2018-12-31'],yearly_returns['2017-12-31'],
                   yearly_returns['2016-12-31'],yearly_returns['2015-12-31'],yearly_returns['2014-12-31'],
                   yearly_returns['2013-12-31'],yearly_returns['2012-12-31'],yearly_returns['2011-12-31'],
                   yearly_returns['2010-12-31']
                  ])
        
        #print(return_series)
    except Exception:
        pass
    return return_series

In [15]:
def computeVolatility(row): 
    volatility = 0  
    try:
        quote = data.get_data_yahoo(row['Symbol'],'01/01/2019',interval='d')
       
        #Compute logarithmic returns
        quote['Log_Ret'] = np.log(quote['Adj Close'] / quote['Adj Close'].shift(1))

        # Compute Volatility using the pandas rolling standard deviation function.  Consider 252 trading days in a year
        quote['Volatility'] = quote['Log_Ret'].rolling(252).std() * np.sqrt(252)
       
        volatility = quote['Volatility'].iloc[-1]
    except Exception:
        pass
    return round(volatility,2)

In [16]:
#computeVolatility()

In [17]:
dataset['close_price'] = dataset.apply(getClosePrice,axis=1)

In [18]:
dataset['risk_score'] = dataset.apply(computeVolatility,axis=1)

In [26]:
dataset[['return_2019', 'return_2018', 'return_2017','return_2016', 'return_2015', 'return_2014','return_2013', 'return_2012', 'return_2011','return_2010']] = dataset.apply(computeReturns, axis=1)

In [27]:
dataset.to_csv('equities.csv')